<a href="https://colab.research.google.com/github/miguelmccormickudg/Gravitational_housing_model/blob/main/modelo_parte_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Cargar el archivo Excel
file_path = '/content/para red social 998 registros.xlsx'
df = pd.read_excel(file_path)

# 1. Contar cuántas veces aparece cada host_name (usaremos transform para mantener dimensiones)
df['host_count_total'] = df.groupby('host_name')['host_name'].transform('count')

# 2. Superhost: 1 si True, 0 si False
df['Superhost_value'] = df['Superhost'].apply(lambda x: 1 if x == True else 0)

# 3. Contar número de cohosts por fila
df['cohosts_count'] = df[['cohosts_1', 'cohosts_2', 'cohosts_3']].notnull().sum(axis=1)

# 4. Asignar valor según ubicación
def location_score(loc):
    if isinstance(loc, str):
        loc = loc.lower()
        if 'chapala' in loc or 'ajijic' in loc:
            return 3
        elif 'guadalajara' in loc:
            return 2
        else:
            return 1
    return 1

df['location_score'] = df['host_location'].apply(location_score)

# 5. Calcular M_host
df['M_host'] = (
    df['host_count_total'] +
    df['Superhost_value'] +
    df['cohosts_count'] +
    df['location_score'] +
    df['host_rating'] +
    df['host_Count'] +
    df['host_years'] +
    df['reviewsCount']
)

# Exportar a CSV
output_path = '/content/M_host_resultado.csv'
df.to_csv(output_path, index=False)

print(f"Archivo guardado en: {output_path}")

Archivo guardado en: /content/M_host_resultado.csv


In [ ]:
import pandas as pd

# Cargar los archivos
df_hosts = pd.read_csv('/content/M_host_resultado.csv')
df_listings = pd.read_excel('/content/para red social 998 registros.xlsx')

# Unir ambos DataFrames por host_name (asumiendo que hay relación)
df = df_listings.merge(df_hosts[['host_name', 'M_host']], on='host_name', how='left')

# 1. room_Category: 2 si private_room, 1 si entire_home
df['room_category_value'] = df['room_Category'].apply(
    lambda x: 2 if x == 'private_room' else (1 if x == 'entire_home' else 0)
)

# 2. petsAllowed: 2 si True, 0 si False
df['petsAllowed_value'] = df['petsAllowed'].apply(lambda x: 2 if x == True else 0)

# 3. canInstantBook: 2 si True, 0 si False
df['canInstantBook_value'] = df['canInstantBook'].apply(lambda x: 2 if x == True else 0)

# 4. Contar amenities no nulos (de las columnas 0 a 9)
amenity_columns = [f'amenities/{i}/title' for i in range(10)]
df['amenities_count'] = df[amenity_columns].notnull().sum(axis=1)

# 5. Calcular M_property
df['M_property'] = (
    df['M_host'] +
    df['room_category_value'] +
    df['Listing_Rating'] +
    df['Reviews'] +
    df['maxGuest'] +
    df['petsAllowed_value'] +
    df['canInstantBook_value'] +
    df['amenities_count']
)

# Exportar a CSV
output_path = '/content/M_property_resultado.csv'
df.to_csv(output_path, index=False)

print(f"Archivo guardado en: {output_path}")

Archivo guardado en: /content/M_property_resultado.csv
